In [1]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [2]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df

In [8]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [10]:
import pandas as pd
def cat_df(sg, sg_hl): 
    df = load_to_df(sg)
    df0 = df.copy()
    
    df = load_to_df(sg_hl)
    df_hl = df.copy()
    
    return pd.merge([df0, df_hl])

In [11]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    del a_df, b_df
    
    return all_df.dropna(axis=0)

In [12]:
flow_pop=multi_columns_merging('상권-추정유동인구', '상권배후지-추정유동인구')
stay_pop=multi_columns_merging('상권-상주인구', '상권배후지-상주인구')
office_pop=multi_columns_merging('상권-직장인구','상권배후지-직장인구')

In [13]:
flow_pop.shape , stay_pop.shape, office_pop.shape

((2020, 1054), (2020, 50), (2020, 44))

In [14]:
def multi_merge(a_df, b_df):
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    return all_df.dropna(axis=0)

In [15]:
people_a=multi_merge(flow_pop,stay_pop)
people=multi_merge(people_a,office_pop)   #유동,상주,직장인구 총 데이터 merge 

In [16]:
targets=['기준_년_분기_코드', '상권_코드', 
         '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수',
         '총_상주인구_수', '남성_상주인구_수', '여성_상주인구_수',
         '총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수']
# total_pop=people_b.filter(regex='|'.join(targets)) #총인구 남자수,여자수 merge 한 값

In [17]:
# 다중 컬럼에서 부분 추출 할때 사용
col_word=['상권_코드','총']    #컬럼 단어 입력하기
def col_value_filter(total_df,col_word):
    result=total_df.filter(regex='|'.join(col_word))
    return result

In [18]:
total_people=col_value_filter(people,targets)

In [19]:
apt_word=['기준_년_분기_코드', '상권_코드','아파트']

In [20]:
apt_df=col_value_filter(people,apt_word)   #아파트 수 추출

In [21]:
ch_df=load_to_df('상권-상권변화지표')
ch_19_df=ch_df[ch_df['기준_년_분기_코드'].str.contains('2019_1|2019_2')]

In [22]:
total_pop_ch=multi_merge(total_people, ch_19_df)   #인구수+지표 총 합한거
total_pop_ch.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,총_상주인구_수,남성_상주인구_수,...,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2019_1,1000001,118266,61268,56997,471903.0,253713.0,218188.0,1806,851,...,206,24345.0,19877.0,4468.0,LH,상권확장,88,57,107,53
1,2019_1,1000002,246619,147358,99261,2186033.0,1237827.0,948202.0,841,383,...,232,13270.0,6153.0,7117.0,HL,상권축소,116,47,107,53
2,2019_1,1000003,144480,80073,64407,1818586.0,1015350.0,803234.0,1214,682,...,2634,44890.0,32318.0,12572.0,HH,정체,113,56,107,53
3,2019_1,1000004,322387,155635,166753,1208957.0,573717.0,635243.0,1453,711,...,46,2196.0,1194.0,1002.0,LH,상권확장,88,57,107,53
4,2019_1,1000005,23622,10781,12848,47589.0,22232.0,25363.0,1694,826,...,388,1820.0,994.0,826.0,LL,다이나믹,85,51,107,53


In [23]:
# targets2=['상권코드','인구','지표']
# col_value_filter(total_pop_ch,targets2)
t_pop_ch=total_pop_ch.iloc[:,:21]

In [24]:
t_pop_ch.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,총_상주인구_수,남성_상주인구_수,...,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표
0,2019_1,1000001,118266,61268,56997,471903.0,253713.0,218188.0,1806,851,...,5455.0,2544.0,2911.0,348,142,206,24345.0,19877.0,4468.0,LH
1,2019_1,1000002,246619,147358,99261,2186033.0,1237827.0,948202.0,841,383,...,11047.0,5624.0,5423.0,486,254,232,13270.0,6153.0,7117.0,HL
2,2019_1,1000003,144480,80073,64407,1818586.0,1015350.0,803234.0,1214,682,...,3512.0,2033.0,1479.0,4822,2188,2634,44890.0,32318.0,12572.0,HH
3,2019_1,1000004,322387,155635,166753,1208957.0,573717.0,635243.0,1453,711,...,9137.0,4521.0,4616.0,104,58,46,2196.0,1194.0,1002.0,LH
4,2019_1,1000005,23622,10781,12848,47589.0,22232.0,25363.0,1694,826,...,6102.0,2958.0,3144.0,1080,692,388,1820.0,994.0,826.0,LL


In [28]:
t_pop_ch['상권_변화_지표'].unique()

array(['LH', 'HL', 'HH', 'LL'], dtype=object)

In [29]:
pd.get_dummies(t_pop_ch, columns=['상권_변화_지표'], drop_first=False)

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,총_상주인구_수,남성_상주인구_수,...,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,2019_1,1000001,118266,61268,56997,471903.0,253713.0,218188.0,1806,851,...,348,142,206,24345.0,19877.0,4468.0,0,0,1,0
1,2019_1,1000002,246619,147358,99261,2186033.0,1237827.0,948202.0,841,383,...,486,254,232,13270.0,6153.0,7117.0,0,1,0,0
2,2019_1,1000003,144480,80073,64407,1818586.0,1015350.0,803234.0,1214,682,...,4822,2188,2634,44890.0,32318.0,12572.0,1,0,0,0
3,2019_1,1000004,322387,155635,166753,1208957.0,573717.0,635243.0,1453,711,...,104,58,46,2196.0,1194.0,1002.0,0,0,1,0
4,2019_1,1000005,23622,10781,12848,47589.0,22232.0,25363.0,1694,826,...,1080,692,388,1820.0,994.0,826.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_2,1001006,309262,149761,159498,1742709.0,852106.0,890599.0,4523,2227,...,504,261,243,1996.0,1108.0,888.0,0,0,1,0
2016,2019_2,1001007,55873,37566,18307,197220.0,129539.0,67683.0,2208,1105,...,462,305,157,1171.0,816.0,355.0,0,0,1,0
2017,2019_2,1001008,390072,189332,200742,3983885.0,1862103.0,2121778.0,2323,1114,...,149,95,54,4362.0,2489.0,1873.0,0,0,0,1
2018,2019_2,1001009,738256,369072,369178,3786483.0,1819086.0,1967396.0,4032,2056,...,543,378,165,6413.0,3834.0,2579.0,0,1,0,0
